# Dataming using pyiron tables

In this example, the data mining capabilities of pyiron using the `PyironTables` class is demonstrated by computing and contrasting the ground state properties of fcc-Al using various force fields.

In [1]:
from pyiron import Project
import numpy as np

In [2]:
pr = Project("potential_scan")

## Uncomment the next line if you want to remove all jobs and start again
# pr.remove_jobs(recursive=True)

## Creating a dummy job to get list of potentials

In order to get the list of available LAMMPS potentials, a dummy job with an Al bulk structure is created

In [3]:
dummy_job = pr.create_job(pr.job_type.Lammps, "dummy_job")
dummy_job.structure = pr.create_ase_bulk("Al")
# Chosing only select potentials to run (you can play with these valuess)
num_potentials = 5
potential_list = dummy_job.list_potentials()[:num_potentials]

## Creating a Murnaghan job for each potential in their respective subprojects

A separate Murnaghan job (to compute equilibrium lattice constant and the bulk modulus) is created and run for every potential

In [4]:
for pot in potential_list:
    pot_str = pot.replace("-", "_")
    # open a subproject within a project
    with pr.open(pot_str) as pr_sub:
        # no need for unique job name if in different subprojects 
        job_name = "murn_Al"
        # Use the subproject to create the jobs
        murn = pr_sub.create_job(pr.job_type.Murnaghan, job_name)
        job_ref = pr_sub.create_job(pr.job_type.Lammps, "Al_ref")
        job_ref.structure = pr.create_ase_bulk("Al", cubic=True)
        job_ref.potential = pot
        job_ref.calc_minimize()
        murn.ref_job = job_ref
        # Some potentials may not work with certain LAMMPS compilations.
        # Therefore, we need to have a little exception handling
        try:
            murn.run()
        except RuntimeError:
            pass

The job murn_Al was saved and received the ID: 1
The job strain_0_9 was saved and received the ID: 2
The job strain_0_92 was saved and received the ID: 3
The job strain_0_94 was saved and received the ID: 4
The job strain_0_96 was saved and received the ID: 5
The job strain_0_98 was saved and received the ID: 6
The job strain_1_0 was saved and received the ID: 7
The job strain_1_02 was saved and received the ID: 8
The job strain_1_04 was saved and received the ID: 9
The job strain_1_06 was saved and received the ID: 10
The job strain_1_08 was saved and received the ID: 11
The job strain_1_1 was saved and received the ID: 12
job_id:  2 finished
job_id:  3 finished
job_id:  4 finished
job_id:  5 finished
job_id:  6 finished
job_id:  7 finished
job_id:  8 finished
job_id:  9 finished
job_id:  10 finished
job_id:  11 finished
job_id:  12 finished
The job murn_Al was saved and received the ID: 13
The job strain_0_9 was saved and received the ID: 14
The job strain_0_92 was saved and received

2020-04-24 13:27:02,849 - pyiron_log - WARNING - Job aborted
2020-04-24 13:27:02,851 - pyiron_log - WARNING - LAMMPS (3 Mar 2020)
Reading data file ...
  orthogonal box = (0 0 0) to (3.91023 3.91023 3.91023)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  4 atoms
  read_data CPU = 0.000882149 secs
ERROR: MEAM library error 3 (src/USER-MEAMC/pair_meamc.cpp:596)
Last command: pair_coeff * * MgAlZn.library.meam Mg Al MgAlZn.parameter.meam Mg Al Zn



The job murn_Al was saved and received the ID: 39
The job strain_0_9 was saved and received the ID: 40
The job strain_0_92 was saved and received the ID: 41
The job strain_0_94 was saved and received the ID: 42
The job strain_0_96 was saved and received the ID: 43
The job strain_0_98 was saved and received the ID: 44
The job strain_1_0 was saved and received the ID: 45
The job strain_1_02 was saved and received the ID: 46
The job strain_1_04 was saved and received the ID: 47
The job strain_1_06 was saved and received the ID: 48
The job strain_1_08 was saved and received the ID: 49
The job strain_1_1 was saved and received the ID: 50
job_id:  40 finished
job_id:  41 finished
job_id:  42 finished
job_id:  43 finished
job_id:  44 finished
job_id:  45 finished
job_id:  46 finished
job_id:  47 finished
job_id:  48 finished
job_id:  49 finished
job_id:  50 finished


If you inspect the job table, you would find that each Murnaghan job generates various small LAMMPS jobs (see column `hamilton`). Some of these jobs might have failed with status `aborted`.

In [5]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,Al4,murn_Al,/murn_Al,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/,2020-04-24 13:25:49.449682,2020-04-24 13:26:10.287283,20.0,pyiron@cmdell17#1#11/11,Murnaghan,0.3.0,None,NaN
1,2,finished,Al4,strain_0_9,/strain_0_9,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:25:50.478025,2020-04-24 13:25:51.243356,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
2,3,finished,Al4,strain_0_92,/strain_0_92,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:25:52.499252,2020-04-24 13:25:53.159412,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
3,4,finished,Al4,strain_0_94,/strain_0_94,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:25:54.326493,2020-04-24 13:25:54.954035,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
4,5,finished,Al4,strain_0_96,/strain_0_96,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:25:56.103131,2020-04-24 13:25:56.726423,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
5,6,finished,Al4,strain_0_98,/strain_0_98,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:25:57.952487,2020-04-24 13:25:58.580900,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
6,7,finished,Al4,strain_1_0,/strain_1_0,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:25:59.746639,2020-04-24 13:26:00.472516,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
7,8,finished,Al4,strain_1_02,/strain_1_02,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:26:01.655404,2020-04-24 13:26:02.284950,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
8,9,finished,Al4,strain_1_04,/strain_1_04,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:26:03.446442,2020-04-24 13:26:04.085234,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0
9,10,finished,Al4,strain_1_06,/strain_1_06,/home/surendralal/,programs/pyiron/notebooks/potential_scan/Al_Mg_Mendelev_eam/murn_Al_hdf5/,2020-04-24 13:26:05.267779,2020-04-24 13:26:05.924324,0.0,pyiron@cmdell17#1,Lammps,0.1,None,1.0


## Analysis using `PyironTables`

The idea now is to go over all finished Murnaghan jobs and extract the equilibrium lattice parameter and bulk modulus, and classify them based of the potential used. Based on the calculated equilibrium lattice constant, it'll be useful to calculate the cohesive energy and the vacancy formation energy as well. The idea is to write functions for this and "apply" it to the table.

### Defining filter functions

Since a project can have thousands if not millions of jobs, it is necessary to "filter" the data and only apply the functions (some of which can be computationally expensive) to only this data. In this example, we need to filter jobs that are finished and are of type `Murnaghan`. This can be done in two ways: using the job table i.e. the entries in the database, or using the job itself i.e. using entries in the stored HDF5 file. Below are examples of filter functions acting on the job and the job table respectively.

In [6]:
# Filtering using the database entries (which are obtained as a pandas Dataframe)
def db_filter_function(job_table):
    # Returns a pandas Series of boolean values (True for entries that have status finished 
    # and hamilton type Murnaghan.)
    return (job_table.status == "finished") & (job_table.hamilton == "Murnaghan")

# Filtering based on the job
def job_filter_function(job):
    # returns a boolean value if the status of the job 
    #is finished and if "murn" is in it's job name 
    return (job.status == "finished") & ("murn" in job.job_name)

Obviously, using the database is faster in this case but sometimes it might be necessary to filter based on some data that are stored in the HDF5 file of the job. The database filter is applied first followed by the job based filter.

### Defining functions that act on jobs

Now we define a set of functions that will be applied on each job to return a certain value. The filtered jobs will be loaded and these functions will be applied on the loaded jobs. The advantage of such functions is that the jobs do not have to be loaded every time such operations are performed. The filtered jobs are loaded once, and then they are passed to these functions to construct the table. Any sort of workflow may be used as a function

In [7]:
# Getting equilibrium lattice parameter from Murnaghan jobs
def get_lattice_parameter(job):
    return job["output/equilibrium_volume"] ** (1/3)

# Getting equilibrium bulk modulus from Murnaghan jobs
def get_bm(job):
    return job["output/equilibrium_bulk_modulus"]

# Getting the potential used in each Murnaghan job
def get_pot(job):
    child = job.project.inspect(job["output/id"][0])
    return child["input/potential/Name"]

# Funtion to create a bulk supercell, run a static LAMMPS calculation
# and return it's  total energy and the number of atoms in the supercell.
def get_bulk_energy_size(job, size=4):
    a = get_lattice_parameter(job)
    el = job["output/structure/species"][0]
    ref_supercell = pr.create_ase_bulk(el, a=a).repeat(size)
    pot = get_pot(job)
    ref_job_name = "rj_{}_s_{}".format(pot, size).replace("-", "_")
    df = pr.job_table()
    if not ref_job_name in df[df.status=="finished"].job.to_list():
        ref_job = pr.create_job(pr.job_type.Lammps, ref_job_name)
        ref_job.structure = ref_supercell
        ref_job.potential = pot
        ref_job.calc_minimize()
        ref_job.run()
    e_ref = pr.inspect(ref_job_name)["output/generic/energy_tot"][-1]
    n_ref = pr.inspect(ref_job_name)["output/generic/positions"].shape[1]
    return e_ref, n_ref 

# A function to calculate the vacancy formation energy for 
# a given supercell size
def get_vac_formation_energy(job, size=4):
    a = get_lattice_parameter(job)
    el = job["output/structure/species"][0]
    ref_supercell = pr.create_ase_bulk(el, a=a).repeat(size)
    def_supercell = ref_supercell[0:-1]
    pot = get_pot(job)
    ref_job_name = "rj_{}_s_{}".format(pot, size).replace("-", "_")
    def_job_name = "dj_{}_s_{}".format(pot, size).replace("-", "_")
    df = pr.job_table()
    if not ref_job_name in df[df.status=="finished"].job.to_list():
        ref_job = pr.create_job(pr.job_type.Lammps, ref_job_name)
        ref_job.structure = ref_supercell
        ref_job.potential = pot
        ref_job.calc_minimize()
        ref_job.run()
    e_ref = pr.inspect(ref_job_name)["output/generic/energy_tot"][-1]
    n_ref = pr.inspect(ref_job_name)["output/generic/positions"].shape[1]
    if not def_job_name in df[df.status=="finished"].job.to_list():
        def_job = pr.create_job(pr.job_type.Lammps, def_job_name)
        def_job.structure = def_supercell
        def_job.potential = pot
        def_job.calc_minimize()
        def_job.run()
    e_def = pr.inspect(def_job_name)["output/generic/energy_tot"][-1]
    n_def = pr.inspect(def_job_name)["output/generic/positions"].shape[1]
    return (e_def - e_ref * (n_def/n_ref))

# Function to calculate the cohesive energy based on the equilibrium 
# lattice parameter
def get_cohesive_energy(job, size=4):
    e_bulk, n_bulk = get_bulk_energy_size(job, size)
    pot = get_pot(job)
    atom_job_name = "aj_{}_s_{}".format(pot, size).replace("-", "_")
    df = pr.job_table()
    if not atom_job_name in df[df.status=="finished"].job.to_list():
        atom_job = pr.create_job(pr.job_type.Lammps, atom_job_name)
        el = job["output/structure/species"][0]
        atom_job.structure = pr.create_atoms(el, 
                                             cell=np.eye(3)* 20, 
                                             scaled_positions=[[0.5, 0.5, 0.5]])
        atom_job.potential = pot
        atom_job.calc_static()
        atom_job.run()
    e_atom = pr.inspect(atom_job_name)["output/generic/energy_tot"][-1]
    return e_bulk / n_bulk - e_atom

### Creating a pyiron table

Now that all the functions are defined, the pyiron table called "table" is created in the following way. This works like a job and can be reloaded at any time.

In [8]:
%%time
# creating a pyiron table
table = pr.create_table("table")

# assigning a database filter function
table.db_filter_function = db_filter_function

# Alternatively/additionally, a job based filter function can be applied 
# (it does the same thing in this case).

#table.filter_function = job_filter_function

# Adding the functions using the labels you like
table.add["a_eq"] = get_lattice_parameter
table.add["bulk_modulus"] = get_bm
table.add["potential"] = get_pot
table.add["vac_formation"] = get_vac_formation_energy

# If you need to pass arguments to the function, 
# they can be defined as lambda funtions. For example, if you
# want to use a larger supercell than the dfault 4x4x4 supercell to calculate
# th vacancy formation energy, you can do this
table.add["vac_formation_larger_cell"] = lambda job: get_vac_formation_energy(job, size=5)
table.add["ecoh"] = get_cohesive_energy

# Running the table to generate the data
table.run(run_again=True)

  0%|          | 0/4 [00:00<?, ?it/s]

The job table was saved and received the ID: 51
The job rj_Al_Mg_Mendelev_eam_s_4 was saved and received the ID: 52
The job dj_Al_Mg_Mendelev_eam_s_4 was saved and received the ID: 53
The job rj_Al_Mg_Mendelev_eam_s_5 was saved and received the ID: 54
The job dj_Al_Mg_Mendelev_eam_s_5 was saved and received the ID: 55
The job aj_Al_Mg_Mendelev_eam_s_4 was saved and received the ID: 56


 25%|██▌       | 1/4 [00:07<00:22,  7.41s/it]

The job rj_Zope_Ti_Al_2003_eam_s_4 was saved and received the ID: 57
The job dj_Zope_Ti_Al_2003_eam_s_4 was saved and received the ID: 58
The job rj_Zope_Ti_Al_2003_eam_s_5 was saved and received the ID: 59
The job dj_Zope_Ti_Al_2003_eam_s_5 was saved and received the ID: 60
The job aj_Zope_Ti_Al_2003_eam_s_4 was saved and received the ID: 61


 50%|█████     | 2/4 [00:15<00:15,  7.72s/it]

The job rj_Al_H_Ni_Angelo_eam_s_4 was saved and received the ID: 62
The job dj_Al_H_Ni_Angelo_eam_s_4 was saved and received the ID: 63
The job rj_Al_H_Ni_Angelo_eam_s_5 was saved and received the ID: 64
The job dj_Al_H_Ni_Angelo_eam_s_5 was saved and received the ID: 65
The job aj_Al_H_Ni_Angelo_eam_s_4 was saved and received the ID: 66


 75%|███████▌  | 3/4 [00:22<00:07,  7.38s/it]

The job rj_2000__Landa_A__Al_Pb__LAMMPS__ipr1_s_4 was saved and received the ID: 67
The job dj_2000__Landa_A__Al_Pb__LAMMPS__ipr1_s_4 was saved and received the ID: 68
The job rj_2000__Landa_A__Al_Pb__LAMMPS__ipr1_s_5 was saved and received the ID: 69
The job dj_2000__Landa_A__Al_Pb__LAMMPS__ipr1_s_5 was saved and received the ID: 70
The job aj_2000__Landa_A__Al_Pb__LAMMPS__ipr1_s_4 was saved and received the ID: 71


100%|██████████| 4/4 [00:28<00:00,  7.17s/it]
2020-04-24 13:27:53,380 - pyiron_log - WARNING - The job table is being loaded instead of running. To re-run use the argument 'run_again=True'


CPU times: user 51.8 s, sys: 11.9 s, total: 1min 3s
Wall time: 29.1 s


The output can now be obtained as a pandas DataFrame

In [9]:
table.get_dataframe()

,job_id,a_eq,bulk_modulus,potential,vac_formation,vac_formation_larger_cell,ecoh
0,1,4.045415,89.015487,Al_Mg_Mendelev_eam,0.667786,0.662386,-3.410657
1,13,4.049946,80.836779,Zope_Ti_Al_2003_eam,0.720309,0.713846,-3.298766
2,25,4.049954,81.040445,Al_H_Ni_Angelo_eam,0.546216,0.531890,-3.360000
3,39,4.031246,78.213776,2000--Landa-A--Al-Pb--LAMMPS--ipr1,0.688258,0.679125,-3.359280


You can now compare the computed equilibrium lattice constants and cohesive energy values for each potential to those computed in the NIST database for Al (fcc phase). https://www.ctcms.nist.gov/potentials/system/Al/#Al.